In [ ]:
!pip show roboflow ultralytics


In [3]:
from roboflow import Roboflow
import os
from ultralytics import YOLO
from datetime import datetime
import cv2
import numpy as np
from pathlib import Path

## Descarregar el dataset

In [ ]:
rf = Roboflow(api_key="5HYzrdPm6LO8xmXDVn3G")
project = rf.workspace("vc").project("matriculas-espanolas")
version = project.version(1)
dataset = version.download("yolov11")                

## Entrenar el model

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)

## Guardar el model

In [ ]:
models_path = "models"
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
model_name = f"model_{fecha_hora_actual}"
model_path = os.path.join(models_path, model_name)
os.makedirs(models_path, exist_ok=True)
model.save(f"{model_path}.pt")

## Carregar el model

In [ ]:
# Configuración
model_path = "./models/model_27_03_2025_19_30.pt"
input_path = "./Matriculas-Españolas-1/valid/images"
output_dir = "./predicted"
os.makedirs(output_dir, exist_ok=True)

# Cargar modelo
model_matr = YOLO(model_path)

## Prediure imagues

In [ ]:
import glob

def predict(model, input_path, output_dir):
    if isinstance(input_path, list):
        image_paths = input_path
    elif os.path.isfile(input_path):
        image_paths = [input_path]
    elif os.path.isdir(input_path):
        image_paths = glob.glob(os.path.join(input_path, '*.jpg')) + \
                      glob.glob(os.path.join(input_path, '*.png')) + \
                      glob.glob(os.path.join(input_path, '*.jpeg'))
    else:
        raise ValueError("Input path no es válido (debe ser imagen, lista o carpeta)")
    
    os.makedirs(output_dir, exist_ok=True)
    
    markdown_content = "# Resultados de predicción\n\n"
    
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error al leer la imagen: {img_path}")
            continue
        
        results = model.predict(img)
        
        img_name = os.path.basename(img_path)
        output_path = os.path.join(output_dir, img_name)
        
        for r in results:
            annotated_img = r.plot()  # Esto devuelve un array numpy
            cv2.imwrite(output_path, annotated_img)
        
        markdown_content += f"## {img_name}\n\n"
        markdown_content += f"![Predicción para {img_name}](../{output_path})\n\n"
        
        for r in results:
            for box in r.boxes:
                markdown_content += f"- Clase: {model.names[int(box.cls)]} | Confianza: {box.conf}\n"
        
        markdown_content += "\n---\n\n"
    
    with open(os.path.join(output_dir, "predictions.md"), "w") as md_file:
        md_file.write(markdown_content)
    
    return markdown_content

if __name__ == "__main__":
    model_path = "./models/model_27_03_2025_19_30.pt"
    input_path = "./Matriculas-Españolas-1/valid/images"  # Puede ser archivo, lista o carpeta
    output_dir = "./predicted"
    
    model_matr = YOLO(model_path)
    
    predict(model_matr, input_path, output_dir)


0: 640x640 1 matricula, 213.3ms
Speed: 1.5ms preprocess, 213.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 matricula, 91.7ms
Speed: 3.4ms preprocess, 91.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 matricula, 81.2ms
Speed: 2.2ms preprocess, 81.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 matricula, 75.8ms
Speed: 2.5ms preprocess, 75.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 matricula, 80.1ms
Speed: 1.4ms preprocess, 80.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 matricula, 103.0ms
Speed: 3.6ms preprocess, 103.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
